<a href="https://colab.research.google.com/github/ssirin/EnergyTransition/blob/ColabCode/ClimateBert_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
## Create a function to analyze all data and return data frame
def climate_analysis(dataframe):
  df = dataframe
  # Define KeyDataset class to extract a specific field from the data
  from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
  from transformers.pipelines.pt_utils import KeyDataset
  import datasets
  from tqdm.auto import tqdm
  import pandas as pd
  from torch.utils.data import Dataset

  class KeyDataset(Dataset):
      def __init__(self, df, key):
          self.df = df
          self.key = key

      def __len__(self):
          return len(self.df)

      def __getitem__(self, idx):
          return {self.key: self.df.loc[idx, self.key]}


## Determine climate related paragraphs

  model_name = "climatebert/distilroberta-base-climate-detector"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

  climate_related = []

  # Iterate through DataFrame rows and apply pipeline
  for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
      # Extract 'Risk Factors' text and put it in a DataFrame
      temp_df = pd.DataFrame({'text': [row['Risk Factors']]})

      # Apply the pipeline
      for out in pipe(KeyDataset(temp_df, 'text'), padding=True, truncation=True):
          # Append results to list (modify as needed based on actual output)
          climate_related.append(out)

  # Add results to new DataFrame column
  df['ClimateRelated'] = climate_related

  return(df)


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/sec_files_energy_risks/sec_risk_2005.txt', sep='\t')
new_df = climate_analysis(df)
new_df.to_csv('/content/drive/MyDrive/SEC_BERT_Results/sec_bert_2005.csv', index = False)